In [1]:
import psycopg2

In [2]:
from pathlib import Path

In [3]:
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Create role

In [44]:
# low risk secrets:
connection = psycopg2.connect(
                                  host    =  '127.0.0.1',
                                  user    =  'postgres',
                                  database=  'postgres',
                                  password=  "DaVinci",
                                  port    = "5432",
                                  connect_timeout=3,
                                )

In [45]:
connection

<connection object at 0x00000213E85896A8; dsn: 'user=postgres password=xxx connect_timeout=3 dbname=postgres host=127.0.0.1 port=5432', closed: 0>

In [46]:
crs = connection.cursor()

In [51]:
crs.execute("SELECT rolname FROM pg_roles")
roles = [item for sublist in crs.fetchall() for item in sublist]
roles

['postgres', 'human', 'mercury']

[]

In [16]:
username = "mercury"
password = "freddie"

psql = f"""
CREATE ROLE {username}
CREATEDB 
LOGIN 
SUPERUSER
PASSWORD '{password}';"""

print(psql)
try:
    crs.execute(psql)
except Exception as e:
    print(e)


CREATE ROLE mercury
CREATEDB 
LOGIN 
SUPERUSER
PASSWORD 'freddie';


In [17]:
crs.execute("SELECT rolname FROM pg_roles")
crs.fetchall()

[('postgres',), ('human',), ('mercury',)]

In [18]:
# COMMIT CHANGES!
connection.commit()

In [19]:
# CLOSE CURSOR
crs.close()

In [20]:
# CLOSE CONNECTION
connection.close()

In [21]:
connection

<connection object at 0x00000213E7AA6D08; dsn: 'user=postgres password=xxx connect_timeout=3 dbname=postgres host=127.0.0.1 port=5432', closed: 1>

# Create Database

In [22]:
# low risk secrets:
connection = psycopg2.connect(
                                  host    =  '127.0.0.1',
                                  user    =  'postgres',
                                  database=  'postgres',
                                  password=  "DaVinci",
                                  port    = "5432",
                                  connect_timeout=3,
                                )
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

In [23]:
cursor = connection.cursor()

In [24]:
name = "queen"
user = "mercury"

cursor.execute(f"CREATE DATABASE '{name}' WITH OWNER='{user}'")

In [25]:
# COMMIT CHANGES!
connection.commit()

In [26]:
cursor.close()

In [27]:
connection.close()

# Delete Database

In [37]:
# low risk secrets:
connection = psycopg2.connect(
                                  host    =  '127.0.0.1',
                                  user    =  'postgres',
                                  database=  'postgres',
                                  password=  "DaVinci",
                                  port    = "5432",
                                  connect_timeout=3,
                                )
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

In [38]:
cursor = connection.cursor()

In [40]:
name = "queen"

# terminate user connections
cursor.execute(f"""SELECT pg_terminate_backend (pid)
FROM pg_stat_activity
WHERE pg_stat_activity.datname = '{name}';""")

cursor.execute(f"DROP DATABASE {name}")

In [41]:
# COMMIT CHANGES!
connection.commit()

In [42]:
cursor.close()

# Restart Server

In [11]:
with psycopg2.connect(
                    # the `postgres` db comes standard issue
                    database="postgres",
                    user="postgres",
                    password="DaVinci",
                    host="127.0.0.1",
                    port="5432",
                    connect_timeout=3,
                  ) as connection:

        crs = connection.cursor()

        # Find where postgres is installed (datadir and pg_ctl in bin)
        crs.execute("select name, setting from pg_settings where name = 'data_directory';")
        data_dir = Path(crs.fetchall()[0][1])
        pg_ctl = data_dir.absolute().parent / 'bin/pg_ctl'

        command = f'''cd {data_dir.absolute().parent}; sudo su postgres -c "{pg_ctl} restart -D {data_dir}"'''
        print("... restarting postgres >>>", command)
        
        crs.close()

... restarting postgres >>> cd /Library/PostgreSQL/9.5; sudo su postgres -c "/Library/PostgreSQL/9.5/bin/pg_ctl restart -D /Library/PostgreSQL/9.5/data"


# Project information

In [28]:
import psycopg2
import psycopg2.extras

In [31]:
with psycopg2.connect(
                    database="queen",
                    user="mercury",
                    password="freddie",
                    host="127.0.0.1",
                    port="5432",
                    connect_timeout=3,
                  ) as connection:
    
    sysid_columns = ['SysId','Name','LastSeen','ClientAddr','UserDefinedClientName']
    sql = f'''SELECT "{'","'.join(sysid_columns)}" FROM public."Sm2SysIdEntry" '''
    
    crs = connection.cursor(cursor_factory = psycopg2.extras.NamedTupleCursor)
    crs.execute(sql)
    sysids = crs.fetchall()
    
    project_columns = ['ProjectName', 'IsLiveCollaborationEnabled', 'SysIds', 'SM_Project_id']
    sql = f'''SELECT "{'","'.join(project_columns)}" FROM public."SM_Project" '''
    
    crs = connection.cursor(cursor_factory = psycopg2.extras.NamedTupleCursor)
    crs.execute(sql)
    projects = crs.fetchall()

In [32]:
sysids

[Record(SysId='f8b156a21693', Name='Philharmonic', LastSeen=1620601067, ClientAddr='127.0.0.1', UserDefinedClientName='')]

In [33]:
projects

[Record(ProjectName='bohemian rhapsody', IsLiveCollaborationEnabled=False, SysIds='', SM_Project_id='70bf16c0-9266-4713-ab9e-9cb607d91014')]

In [24]:
projects[0].SysIds.split(',')

['3C22FB9CB594']

In [42]:
def user_in_project(user):
    """ For a given user, what project are they in, if at all? """
    
    with psycopg2.connect(
                    # the `postgres` db comes standard issue
                    database="",
                    user="",
                    password="",
                    host="127.0.0.1",
                    port="5432",
                    connect_timeout=3,
                  ) as connection:
    
        sysid_columns = ['SysId','Name','LastSeen','ClientAddr','UserDefinedClientName']
        sql = f'''SELECT "{'","'.join(sysid_columns)}" FROM public."Sm2SysIdEntry" '''

        crs = connection.cursor(cursor_factory = psycopg2.extras.NamedTupleCursor)
        crs.execute(sql)
        sysids = crs.fetchall()

        project_columns = ['ProjectName', 'IsLiveCollaborationEnabled', 'SysIds', 'SM_Project_id']
        sql = f'''SELECT "{'","'.join(project_columns)}" FROM public."SM_Project" '''

        crs = connection.cursor(cursor_factory = psycopg2.extras.NamedTupleCursor)
        crs.execute(sql)
        projects = crs.fetchall()
    
    if len(sysids) == 0:
        # if there are no users in database this won't work...
        return
    
    user_ip = user['ip']
    
    if user['name'] == 'Server':
        # remap to home address if looking at server user
        #  because resolve reports the home address, not first_ip
        user_ip = '127.0.0.1' 

    # get sysid of user with user_ip
    for select in sysids:
        
        if user_ip == select.ClientAddr:
            # carry to after the loop
            user_sysid = select.SysId
            break
            
    else:
        print(f">>> User with ip {user_ip} was not found in Resolve database")
        return

    for project in projects:

        if project.SysIds != '':
            # Someone's in there!
            
            user_ids = project.SysIds.split(',')

            # Map to user
            
            for sysid in user_ids:
                
                if user_sysid == sysid:
                    
                    print(f">>> {user['name']} is in {project.ProjectName}")
                    
                    return project.ProjectName

In [43]:
user_in_project({'name':'Server', 'ip':'7.0.0.1'})

>>> Server is in Hilbert Hotel


'Hilbert Hotel'

In [44]:
user_in_project({'name':'Derek', 'ip':'7.0.0.7'})

>>> User with ip 7.0.0.7 was not found in Resolve database
